# 🚀 Advanced Applications

### Config

In [2]:
from IPython.display import display, Markdown
from config import OPENAI_API_KEY
import openai
import os

In [3]:
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY

# Define function for printing long strings as markdown
md_print = lambda text: display(Markdown(text))

## Introduction

In this section we go very powerful, more advanced applications of prompt engineering.

## LLMs Using Tools

**MRKL Systems (Modular Reasoning, Knowledge and Language, pronounced "miracle")** are a neuro-symbolic architecture that combine LLMs and enternal tools to solve complex problems. This can include calculators, APIs, custom functions, anything can be a tool!!!

The MRKL systems uses an LLM as a router to determine when an external tool is needed. Here are a few examples of how a MRKL system works in action:

    # Example 1: Calculator

    What is 100*100?
    CALCULATOR[100*100]


    # Example 2: Stock Price

    What is the price of Apple stock right now?
    The current price is DATABASE[SELECT price FROM stock WHERE company = "Apple" AND time = "now"].
    

    # Example 3: Weather

    What is the weather like in New York?
    The weather is WEATHER_API[New York].


Let's use Langchain to replicate the calculator use-case, here we use a ReAct agent to help us accomplish the tool integration MRKL demonstrates:

In [5]:
from langchain import LLMMathChain, OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# Initialize LLM
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Initialize MathChain
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# Define tools
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

# Initialize agent
mrkl = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Run Query to agent
mrkl.run("What is two times thirty-four divided by three?")



> Entering new AgentExecutor chain...
 I need to divide two times thirty-four by three.
Action: Calculator
Action Input: 2 * 34 / 3

> Entering new LLMMathChain chain...
2 * 34 / 3
```python
print(2 * 34 / 3)
```

Answer: 22.666666666666668

> Finished chain.

Observation: Answer: 22.666666666666668

Thought: I now know the final answer
Final Answer: 22.666666666666668

> Finished chain.


'22.666666666666668'

## LLMs that Reason and Act

**ReAct (reason, act)** is a paradigm for enabling language models to solve complex tasks uitilizing natural language reasoning. They reason about the actions they can perform. The workflow for the Language model in the ReAct paradigm is as follows:

1. Thought (Reasoning about the question)
2. Act (Perform an action)
3. Observation (Observe the output of the action)

These steps are looped until a "conclusion" is reached and the model is done with the task.

This is similar to the classic RL loops of state (Action, Reward, State) which is mentioned in the ReAct paper. ReAct performs well against CoT and other methods, and is the default framework for Agent reasoning in popular LLM frameworks like Langchain.

In [7]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

# Create tools
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

# Initialize LLM
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Initialize Agent
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

# Ask a question to our ReAct Agent
question = "When was GPT-2 released?"
react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search GPT-2 and find when it was released.
Action: Search[GPT-2]
Observation: Could not find [GPT-2]. Similar: ['GPT-2', 'GPT-3', 'GPT-4', 'Generative pre-trained transformer', 'OpenAI', 'ChatGPT', 'GPT-J', 'AI Dungeon', 'Large language model', 'GPT2']
Thought: GPT-2 is a type of language model, so I can search GPT-2 language model and find when it was released.
Action: Search[GPT-2 language model]
Observation: Generative Pre-trained Transformer 2 (GPT-2)  is an open-source artificial intelligence large language model created by OpenAI in February 2019. GPT-2 translates text, answers questions, summarizes passages, and generates text output on a level that, while sometimes indistinguishable from that of humans, can become repetitive or nonsensical when generating long passages. It is a general-purpose learner; it was not specifically trained to do any of these tasks, and its ability to perform them is an extension of its gener

'February 2019, November 2019'

## Code as Reasoning

**Program-aided Language Models (PAL)** are another example of a MRKL system. The main difference is that they excecute code as opposed to external tools to solve problems. The workflow is similar to that of CoT, but PAL uses code as the intermediate reasoning, while CoT uses Natural Language.

PAL actually uses some natural language reasoning in the form of comments within its code. Let's use Langchain to create a PAL Agent that will answer our math question using a few-shot prompt:

In [14]:
# Initialize LLM
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Set prompt
MATH_PROMPT = '''
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?

# solution in Python:
"""There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?"""
computers_initial = 9
computers_per_day = 5
num_days = 4  # 4 days between monday and thursday
computers_added = computers_per_day * num_days
computers_total = computers_initial + computers_added
result = computers_total
return result


Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?

# solution in Python:
"""Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?"""
toys_initial = 5
mom_toys = 2
dad_toys = 2
total_received = mom_toys + dad_toys
total_toys = toys_initial + total_received
result = total_toys


Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?

# solution in Python:
"""Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
jason_lollipops_initial = 20
jason_lollipops_after = 12
denny_lollipops = jason_lollipops_initial - jason_lollipops_after
result = denny_lollipops

Q: If a bakery has 20 loaves of bread and sells 3 loaves every hour, how many loaves will be left after the bakery is open for 5 hours?

# solution in Python:
'''

# Execute LLM to generate python solution
llm_out = llm(MATH_PROMPT.format(question=question))
print(llm_out)

# Execute python code
exec(llm_out)
print(result)

"""If a bakery has 20 loaves of bread and sells 3 loaves every hour, how many loaves will be left after the bakery is open for 5 hours?"""
loaves_initial = 20
loaves_sold_per_hour = 3
hours_open = 5
loaves_sold = loaves_sold_per_hour * hours_open
loaves_left = loaves_initial - loaves_sold
result = loaves_left
5
